## Problem Statement
Type: Binary Classification<br>
An organization wants to predict who possible defaulters are for the consumer loans product. They have data about historic customer behavior based on what they have observed. Hence when they acquire new customers they want to predict who is more risky and who is not.

In [1]:
#Basic and most important libraries
import pandas as pd , numpy as np
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from collections import Counter
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly

In [2]:
#Classifiers
from sklearn.ensemble import AdaBoostClassifier , GradientBoostingClassifier , VotingClassifier , RandomForestClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import GaussianNB
# from xgboost import plot_importance
# from xgboost import XGBClassifier
from sklearn.svm import SVC


In [3]:
#Model evaluation tools
from sklearn.metrics import classification_report , accuracy_score , confusion_matrix
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score,roc_curve

In [4]:
#Data processing functions
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [5]:
data = pd.read_csv("../input/loandata/Training Data.csv")
output=pd.read_csv('../input/loandata/Test Data.csv')
#data=data.set_index('Id')
data=pd.concat([data,output])
data.head(5)

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag,id
0,1.0,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0.0,NaN
1,2.0,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0.0,NaN
2,3.0,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0.0,NaN
3,4.0,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1.0,NaN
4,5.0,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1.0,NaN


In [6]:
data["married"] = le.fit_transform(data["married"])
data["house_ownership"] = le.fit_transform(data["house_ownership"])
data["car_ownership"] = le.fit_transform(data["car_ownership"])
data["profession"] = le.fit_transform(data["profession"])
data["city"] = le.fit_transform(data["city"])
data["state"] = le.fit_transform(data["state"])

#data = pd.get_dummies(data)
data.head(5)

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag,id
0,1.0,1303835,23,3,1,2,0,50,262,17,3,13,0.0,NaN
1,2.0,7574516,40,10,1,2,0,63,236,18,9,13,0.0,NaN
2,3.0,3991815,66,4,0,2,0,68,8,15,4,10,0.0,NaN
3,4.0,6256451,41,2,1,2,1,63,55,21,2,12,1.0,NaN
4,5.0,5768871,47,11,1,2,0,18,312,27,3,14,1.0,NaN


In [7]:
def normalise_column(li):
    for column in li:
        min= np.min(data[column])
        max= np.max(data[column])
        data[column]=data[column].apply(lambda x: (x-min)/(max-min))
normalise_column(['income', 'age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'profession', 'city', 'state', 'current_job_years',
       'current_house_years'])

In [8]:
from matplotlib import pyplot as plt

In [9]:
output=data[data['id']<30000]

In [10]:
data=data[data['Id']<270000]

In [11]:
data=pd.concat([data,data[data['risk_flag']==1],data[data['risk_flag']==1],data[data['risk_flag']==1],data[data['risk_flag']==1]]
)

In [12]:
len(data)

375984

In [13]:
Features=['income', 
          'age', 'experience', 'married', 
          'house_ownership', 'car_ownership', 
          'profession', 'city', 'state', 'current_job_years', 
          'current_house_years', 'risk_flag']

In [14]:
data=data[Features]

In [15]:
#data.sort_values(by='Id').to_csv('balanced_data.csv')

In [16]:
#data[Features].to_csv('balanced_data.csv')

In [17]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
mm_scaler.fit_transform(data)
mm_scaler.transform(data)

array([[0.1294868 , 0.03448276, 0.15      , ..., 0.21428571, 0.75      ,
        0.        ],
       [0.75720597, 0.32758621, 0.5       , ..., 0.64285714, 0.75      ,
        0.        ],
       [0.39856389, 0.77586207, 0.2       , ..., 0.28571429, 0.        ,
        0.        ],
       ...,
       [0.17882578, 0.44827586, 0.1       , ..., 0.14285714, 0.5       ,
        1.        ],
       [0.31752734, 0.53448276, 0.1       , ..., 0.14285714, 0.        ,
        1.        ],
       [0.81391589, 0.67241379, 0.5       , ..., 0.64285714, 0.75      ,
        1.        ]])

In [18]:
# data.to_csv('train.csv')

In [19]:
data.describe()

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
count,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000,375984.000000
mean,0.498414,0.493558,0.495098,0.903528,0.949937,0.291874,0.517269,0.494451,0.486924,0.448534,0.498101,0.412198
std,0.289549,0.296056,0.301554,0.295238,0.189426,0.454625,0.291271,0.291482,0.323987,0.262965,0.349854,0.492231
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.027778,0.000000,0.000000,0.000000
25%,0.245981,0.224138,0.250000,1.000000,1.000000,0.000000,0.260274,0.243976,0.194444,0.214286,0.250000,0.000000
50%,0.498971,0.500000,0.500000,1.000000,1.000000,0.000000,0.534247,0.484940,0.472222,0.428571,0.500000,0.000000
75%,0.750281,0.758621,0.750000,1.000000,1.000000,1.000000,0.780822,0.746988,0.750000,0.642857,0.750000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
Features_train=['income', 'age', 'experience', 'married', 'house_ownership',
       'car_ownership', 'profession', 'city', 'state', 'current_job_years',
       'current_house_years']

It seems Application income and Loan Amount is correlated , also Coapplication income correlated with Loan Aount then 
Credit history is corrleated with Loan Status

In [21]:
feat=[ 'income', 'age', 'experience', 'married',
       'house_ownership', 'car_ownership', 'profession', 'city', 'state',
       'current_job_years', 'current_house_years']

In [22]:
new_data = data.copy()
X = data[Features_train]
y = data["risk_flag"]
counter = Counter(y)
print("Counter: ",counter)

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25,random_state=0)
#Check mean is 0 and Standard deviation is 1
print("After Standardization\nMean ",np.mean(X_train),"Standard Deviation ",np.std(X_train),"\n")

#Voting ensemble mathod. Combining all tree based algorithms.
models = []
models.append(("RF",RandomForestClassifier()))
models.append(("DT",DecisionTreeClassifier()))
models.append(("ADB",AdaBoostClassifier()))
models.append(("GB",GradientBoostingClassifier()))

ensemble = VotingClassifier(estimators=models,voting='soft')
ensemble.fit(X_train,y_train)
y_pred = ensemble.predict(X_test) 
print(classification_report(y_pred,y_test))
print("Voting Ensemble:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("Voting Ensemble:> roc_auc" ,roc)


Counter:  Counter({0.0: 221004, 1.0: 154980})
After Standardization
Mean  income                 0.498491
age                    0.493850
experience             0.494680
married                0.903560
house_ownership        0.949759
car_ownership          0.292590
profession             0.517471
city                   0.494999
state                  0.487190
current_job_years      0.448202
current_house_years    0.498159
dtype: float64 Standard Deviation  income                 0.289519
age                    0.296146
experience             0.301606
married                0.295194
house_ownership        0.189946
car_ownership          0.454952
profession             0.291488
city                   0.291647
state                  0.323775
current_job_years      0.262932
current_house_years    0.349920
dtype: float64 

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94     49132
         1.0       1.00      0.86      0.92     44864

    ac

In [23]:
data['1']=ensemble.predict_proba(data[Features_train])[:,1]
output['1']=ensemble.predict_proba(output[Features_train])[:,1]

In [24]:

Model1 = RandomForestClassifier(n_estimators=1000,random_state=0,n_jobs=1000,max_depth=70,bootstrap=True)
Model1.fit(X_train,y_train)
y_pred = Model1.predict(X_test)
print(classification_report(y_pred,y_test))
print("RandomForestClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("RandomForestClassifier:> roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95     50710
         1.0       1.00      0.89      0.94     43286

    accuracy                           0.95     93996
   macro avg       0.96      0.94      0.95     93996
weighted avg       0.95      0.95      0.95     93996

RandomForestClassifier:> 0.9483169496574322
RandomForestClassifier:> roc_auc 0.9559231792543696


In [25]:
data['2']=Model1.predict_proba(data[Features_train])[:,1]
output['2']=Model1.predict_proba(output[Features_train])[:,1]

In [26]:
print("training started")
Model2 = GradientBoostingClassifier()
Model2.fit(X_train,y_train)
y_pred = Model2.predict(X_test)
print(classification_report(y_pred,y_test))
print("GradientBoostingClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

training started
              precision    recall  f1-score   support

         0.0       0.96      0.62      0.75     86059
         1.0       0.15      0.71      0.24      7937

    accuracy                           0.63     93996
   macro avg       0.55      0.66      0.50     93996
weighted avg       0.89      0.63      0.71     93996

GradientBoostingClassifier:> 0.6252074556364101
roc_auc 0.5522832785532058


In [27]:
data['3']=Model2.predict_proba(data[Features_train])[:,1]
output['3']=Model2.predict_proba(output[Features_train])[:,1]

In [28]:

Model3 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
 max_features=1.0, max_leaf_nodes=10,
  min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.10,
  random_state=27, splitter='best')
Model3.fit(X_train,y_train)
y_pred = Model3.predict(X_test)
print(classification_report(y_pred,y_test))
print("DecisionTreeClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       1.00      0.59      0.74     93996
         1.0       0.00      0.00      0.00         0

    accuracy                           0.59     93996
   macro avg       0.50      0.29      0.37     93996
weighted avg       1.00      0.59      0.74     93996

DecisionTreeClassifier:> 0.5897697774373377
roc_auc 0.5


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
data['4']=Model3.predict_proba(data[Features_train])[:,1]
output['4']=Model3.predict_proba(output[Features_train])[:,1]

In [30]:

R=RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1,max_depth=70,bootstrap=True)
Model4 = AdaBoostClassifier(base_estimator=R,n_estimators=10)
Model4.fit(X_train,y_train)
y_pred = Model4.predict(X_test)
print(classification_report(y_pred,y_test))
print("AdaBoostClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95     50771
         1.0       1.00      0.89      0.94     43225

    accuracy                           0.95     93996
   macro avg       0.96      0.94      0.95     93996
weighted avg       0.95      0.95      0.95     93996

AdaBoostClassifier:> 0.948859525937274
roc_auc 0.9563634321765835


In [31]:
data['5']=Model4.predict_proba(data[Features_train])[:,1]
output['5']=Model4.predict_proba(output[Features_train])[:,1]

In [32]:
Model5 = LinearDiscriminantAnalysis()
Model5.fit(X_train,y_train)
y_pred = Model5.predict(X_test)
print(classification_report(y_pred,y_test))
print("LinearDiscriminantAnalysis:>",accuracy_score(y_pred,y_test),"\n")
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)


              precision    recall  f1-score   support

         0.0       1.00      0.59      0.74     93738
         1.0       0.00      0.72      0.01       258

    accuracy                           0.59     93996
   macro avg       0.50      0.65      0.38     93996
weighted avg       1.00      0.59      0.74     93996

LinearDiscriminantAnalysis:> 0.5909613175028725 

roc_auc 0.5017404420079647


In [33]:
data['6']=Model5.predict_proba(data[Features_train])[:,1]
output['6']=Model5.predict_proba(output[Features_train])[:,1]

In [34]:
KNN = KNeighborsClassifier(leaf_size=1,p=2,n_neighbors=7)
KNN.fit(X_train,y_train)
y_pred = KNN.predict(X_test)
print(classification_report(y_pred,y_test))
print("KNeighborsClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       0.86      0.93      0.90     51076
         1.0       0.91      0.82      0.86     42920

    accuracy                           0.88     93996
   macro avg       0.89      0.88      0.88     93996
weighted avg       0.88      0.88      0.88     93996

KNeighborsClassifier:> 0.8824630835354695
roc_auc 0.8871534712826483


In [35]:
data['7']=KNN.predict_proba(data[Features_train])[:,1]
output['7']=KNN.predict_proba(output[Features_train])[:,1]

In [36]:

Model7 = GaussianNB()
Model7.fit(X_train,y_train)
y_pred = Model7.predict(X_test)
print(classification_report(y_pred,y_test))
print("GaussianNB:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       0.78      0.61      0.68     71316
         1.0       0.27      0.46      0.34     22680

    accuracy                           0.57     93996
   macro avg       0.52      0.53      0.51     93996
weighted avg       0.66      0.57      0.60     93996

GaussianNB:> 0.5701093663560152
roc_auc 0.5244481992467708


In [37]:
data['8']=Model7.predict_proba(data[Features_train])[:,1]
output['8']=Model7.predict_proba(output[Features_train])[:,1]

In [38]:
Model8 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Model8.fit(X_train,y_train)
y_pred = Model8.predict(X_test)
print(classification_report(y_pred,y_test))
print("Logistic Regression:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       1.00      0.59      0.74     93736
         1.0       0.00      0.72      0.01       260

    accuracy                           0.59     93996
   macro avg       0.50      0.65      0.38     93996
weighted avg       1.00      0.59      0.74     93996

Logistic Regression:> 0.5909613175028725
roc_auc 0.501744389403174


In [39]:
data['9']=Model8.predict_proba(data[Features_train])[:,1]
output['9']=Model8.predict_proba(output[Features_train])[:,1]

In [40]:
Model9 = AdaBoostClassifier(n_estimators=300)
Model9.fit(X_train,y_train)
y_pred = Model9.predict(X_test)
print(classification_report(y_pred,y_test))
print("AdaBoostClassifier:>",accuracy_score(y_pred,y_test))
roc=roc_auc_score(y_test,y_pred)
print("roc_auc" ,roc)

              precision    recall  f1-score   support

         0.0       0.92      0.62      0.74     82793
         1.0       0.17      0.60      0.27     11203

    accuracy                           0.61     93996
   macro avg       0.55      0.61      0.50     93996
weighted avg       0.83      0.61      0.68     93996

AdaBoostClassifier:> 0.6139942125196817
roc_auc 0.5471426397574386


In [42]:
data['10']=Model9.predict_proba(data[Features_train])[:,1]
output['10']=Model9.predict_proba(output[Features_train])[:,1]

In [43]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

# define model
model = Sequential()
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
x=data[['1','2','3','4','5','6','7','8','9','10']]
y=data['risk_flag']
y=to_categorical(y)

In [45]:
history = model.fit(x,y,validation_split=0.2,batch_size=256,epochs=50)

Epoch 1/50
1175/1175 [==============================] - 3s 2ms/step - loss: 0.3350 - accuracy: 0.8799 - val_loss: 0.1804 - val_accuracy: 0.9884
Epoch 2/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1454 - accuracy: 0.9376 - val_loss: 0.1943 - val_accuracy: 0.9821
Epoch 3/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1453 - accuracy: 0.9367 - val_loss: 0.1516 - val_accuracy: 0.9910
Epoch 4/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1432 - accuracy: 0.9380 - val_loss: 0.1842 - val_accuracy: 0.9877
Epoch 5/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1436 - accuracy: 0.9375 - val_loss: 0.2106 - val_accuracy: 0.9784
Epoch 6/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1440 - accuracy: 0.9373 - val_loss: 0.1781 - val_accuracy: 0.9889
Epoch 7/50
1175/1175 [==============================] - 2s 2ms/step - loss: 0.1429 - accuracy: 0.9376 - val_loss: 0.1755 - val_accuracy:

In [47]:
output['risk_flag']=model.predict(output[
    ['1','2','3','4','5','6','7','8','9','10']
])

In [48]:
output[['id','risk_flag']].to_csv('submission13.csv')